In [ ]:
'''OT1 -Single channel- Making up of main reagent stock solution (2 maximum) using a defined solvent and transfer onto 3mL fluidx vials'''

from opentrons import robot, containers, instruments

robot.head_speed(x=21000, y=21000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

robot.reset()
# CSV file data
stock_reagent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\stock_reagents.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\urea_phip_triple_reaction\csv\solvents.csv")

"""Function that does two liquid handling transfers. First it will dilute a solid reagent in a big trough to the right concentration.
(Up to 2 reagents). Second, it will transfer the reagent from the big trough to the Fluix 24 vial rack, using the volume from the csv file
It requires 2 csv files. The first is the solvent csv, the second the custom made stock reagent csv."""


def stock_solution(amine, solvent):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough4row = containers.load("trough-12row", "C2")
    destination_stock = containers.load("FluidX_24_5ml", "A1", "stock")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )

    id_header = "CPD ID"
    solvent = "DMA"
    stock_sol1 = "stock reagent 1"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (uL)"
    volume_per_vial = "Volume to dispense"

    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
        if x == stock_sol1:
            stock_sol1_loc = solvent_df[location_header].tolist()[i]
            stock_sol1_volume = solvent_df[volume_stock_header].tolist()[i]

    # Using the desired solvent, dilution of reagents 1 and/or 2 to the desired conc, in the big trough
    p1000.pick_up_tip()
    p1000.transfer([stock_sol1_volume], source_trough4row.wells(solvent_location),
                   source_trough4row.wells(stock_sol1_loc).top(-5), new_tip='never')
    p1000.drop_tip()

    robot.pause()

    # Reagents 1 and/or 2 are transfered to the 24 Fluidx vial rack.
    for i, x in enumerate(stock_reagent_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [stock_reagent_df[volume_per_vial].tolist()[i]]
        stock_id = stock_reagent_df[id_header].tolist()[i]
        if stock_id == stock_sol1:
            p1000.pick_up_tip()
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol1_loc),
                               destination_stock.wells(destination_location).top(-5), new_tip='never')
            p1000.drop_tip()
    robot.home()
stock_solution(stock_reagent_df, solvent_df)

robot.commands()


In [83]:
"""This protocol transfers the main reactant onto the 96 reaction rack. It takes information from 1 csv, stock_reagents. It takes sequentially the reactant from the 24 fluidx racks to the 96 plate"""

from opentrons import robot, containers, instruments
robot.head_speed(x=18000, y=18000, z=5000, a=250, b=250)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

robot.reset()
# CSV file data
reaction_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\stock_reagents.csv")

def mainReactant_transfer(reaction):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough12row = containers.load("trough-12row", "C1")
    location_stock = containers.load("FluidX_24_5ml", "A1", "int")
    reaction_rack = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    location_header = "Location"
    nb_reaction_header = "Number reaction"
    volume_per_reaction_header = "Volume_stock per reaction"
    volume_max_header = "Volume max per vial"
    nb_reactions = int(reaction_df[nb_reaction_header].tolist()[0])
    volume_per_reaction = int(reaction_df[volume_per_reaction_header].tolist()[0])
    
    reaction_counter = 0
    """In each well, the volume per reaction of reaction is dispensed, successively in the logical order ("A1", "A2"...). When the maximum amount of reactant in one vial is taken out, the following
    vial in the fluidx rack is used. The transfer stops when the reactant is dispensed in all the wells of the 96 plate or all the wells used for the batch."""
    
    
    for index, value in enumerate(reaction_df[location_header].tolist()):
        if reaction_counter < nb_reactions:
            nb_reaction_per_vial = int(float(reaction_df[volume_max_header].tolist()[index]) // volume_per_reaction)
            source_location = value
            
            if nb_reactions - reaction_counter < nb_reaction_per_vial:
                if nb_reactions - reaction_counter == 1:
                    p1000.distribute(volume_per_reaction, location_stock.wells(source_location),
                                     reaction_rack.wells(reaction_counter).top())
                else:
                    p1000.distribute(volume_per_reaction, location_stock.wells(source_location),
                                     [x.top() for x in reaction_rack.wells(reaction_counter, to=nb_reactions)])
            else:
                p1000.distribute(volume_per_reaction, location_stock.wells(source_location), [x.top() for x in
                                                                                              reaction_rack.wells(
                                                                                                  reaction_counter,
                                                                                                  to=reaction_counter + nb_reaction_per_vial - 1)])
            reaction_counter = reaction_counter + nb_reaction_per_vial
    robot.home()
    print(nb_reactions, volume_per_reaction, nb_reaction_per_vial, reaction_counter)
mainReactant_transfer(reaction_df)
robot.commands()

66 121 10 55


['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 998.0 at <Deck><Slot A1><Container int><Well A1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well B1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well D1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well E1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well F1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well G1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well H1>',
 'Blowing out at <Deck><Slot C3><Container point><Well A1>',
 'Aspirating 877.0 at <Deck><Slot A1><Container int><Well A1>',
 'Dispensing 121.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A2>',
 'Dispensing 121.0 at <Deck><Slot D1><Container St

In [71]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
reactants_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\carboxylic_acids.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\solvents.csv")

robot.reset()


def reactants_transfer(reactants, solvent):
    # Deck setup

    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    tiprack_1000_2 = containers.load("tiprack-1000ul-H", "D3")
    #tiprack_1000_3 = containers.load("tiprack-1000ul-H", "B1")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_reactants_1 = containers.load("FluidX_24_5ml", "A1", "R_1")
    rack_stock_reactants_2 = containers.load("FluidX_24_5ml", "A2", "R_2")
    rack_stock_reactants_3 = containers.load("FluidX_24_5ml", "B2", "R_3")
    rack_stock_reactants_4 = containers.load("FluidX_24_5ml", "B1", "R_4")
    reaction_rack = containers.load("StarLab_96_tall", "C1")
    trash = containers.load("point", "C3")

    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000, tiprack_1000_2],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )

    location_header = "Location"
    volume_per_reaction_header = "volume_per_vial"
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMA"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    solvent_location_header = "Location_trough"
    solvent_volume_header = "Volume to dispense (uL)"
    counter = 0
    for i, x in enumerate(reactants_df[rack_ID_header].tolist()):
        volume_per_reaction = [reactants_df[volume_per_reaction_header].tolist()[i]]
        source_location = reactants_df[location_header].tolist()[i]

        if x == "":
            #print('null')
            break
        if x == rack_1:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_1.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
        if x == rack_2:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_2.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
        if x == rack_3:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_3.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
        if x == rack_4:
            p1000.pick_up_tip()
            p1000.transfer(volume_per_reaction, rack_stock_reactants_4.wells(source_location),
                           reaction_rack.wells(i + 2).top(-5), new_tip='never')
        counter += 1
    # p1000.pick_up_tip()
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            vol_to_add = [solvent_df[solvent_volume_header].tolist()[i]]
            solvent_loc = [solvent_df[solvent_location_header].tolist()[i]]

    p1000.distribute(vol_to_add, source_trough4row.wells(solvent_loc),
                     [x.top() for x in reaction_rack.wells(0, to=counter + 1)])
    #print(counter)
    robot.home()

reactants_transfer(reactants_df, solvent_df)
robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container R_1><Well A1>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well C1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well B1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container R_1><Well A2>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well D1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well C1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container R_1><Well A3>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well E1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well D1>',
 'Aspirating 132.0 at <Deck><Slot A1><Container R_1><Well A4>',
 'Dispensing 132.0 at <Deck><Slot C1><Container StarLab_96_tall><Well F1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A2>',
 'Aspirating 132.0 at <Deck><Slot A1><Container R_1><We

In [67]:
from opentrons import robot, containers, instruments

robot.head_speed(x=18000, y=18000, z=5000, a=700, b=700)


class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list


class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length


# Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df


# CSV file data
reactants_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\carboxylic_acids.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\solvents.csv")

robot.reset()

def stock_solution_reactants(reactants, solvent):
    # Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough4row = containers.load("trough-12row", "C2")
    rack_stock_reactants_1 = containers.load("FluidX_24_5ml", "A1", "R_1")
    rack_stock_reactants_2 = containers.load("FluidX_24_5ml", "A2", "R_2")
    rack_stock_reactants_3 = containers.load("FluidX_24_5ml", "A3", "R_3")
    rack_stock_reactants_4 = containers.load("FluidX_24_5ml", "B1", "R_4")
    trash = containers.load("point", "C3")
    # Pipettes SetUp
    p1000 = instruments.Pipette(
        name='eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    rack_ID_header = "Rack ID"
    id_header = "CPD ID"
    solvent = "DMA"
    rack_1 = "24_rack1"
    rack_2 = "24_rack2"
    rack_3 = "24_rack3"
    rack_4 = "24_rack4"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (exp) at 0.5M"

    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
    for i, x in enumerate(reactants_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [reactants_df[volume_stock_header].tolist()[i]]
        reactants_id = reactants_df[rack_ID_header].tolist()[i]
        if reactants_id == "":
            print ('null')
            break
        print (rack_ID_header, reactants_id, vol_to_dispense)
        p1000.pick_up_tip()
        if reactants_id == rack_1:
            print ('rack1')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_1.wells(destination_location).top(-5), new_tip='never')
        if reactants_id == rack_2:
            print ('rack2')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_2.wells(destination_location).top(-5), new_tip='never')
        if reactants_id == rack_3:
            print ('rack3')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_3.wells(destination_location).top(-5), new_tip='never')
        if reactants_id == rack_4:
            print ('rack4')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(solvent_location),
                               rack_stock_reactants_4.wells(destination_location).top(-5), new_tip='never')
        p1000.drop_tip()
    robot.home()
stock_solution_reactants(reactants_df, solvent_df)

robot.commands()

Rack ID 24_rack1 ['1443.742099']
rack1
Rack ID 24_rack1 ['1337.914743']
rack1
Rack ID 24_rack1 ['1536.755067']
rack1
Rack ID 24_rack1 ['1620.708196']
rack1
Rack ID 24_rack1 ['1804.132662']
rack1
Rack ID 24_rack1 ['1635.369622']
rack1
Rack ID 24_rack1 ['1787.585527']
rack1
Rack ID 24_rack1 ['1850.95455']
rack1
Rack ID 24_rack1 ['1841.054313']
rack1
Rack ID 24_rack1 ['1475.169751']
rack1
Rack ID 24_rack1 ['1811.909949']
rack1
Rack ID 24_rack1 ['1881.541389']
rack1
Rack ID 24_rack1 ['1966.356863']
rack1
Rack ID 24_rack1 ['1782.54783']
rack1
Rack ID 24_rack1 ['1903.199344']
rack1
Rack ID 24_rack1 ['1818.013686']
rack1
Rack ID 24_rack1 ['1944.074567']
rack1
Rack ID 24_rack1 ['1873.184555']
rack1
Rack ID 24_rack1 ['1837.999082']
rack1
Rack ID 24_rack1 ['1889.07112']
rack1
Rack ID 24_rack1 ['1794.401101']
rack1
Rack ID 24_rack1 ['1894.359587']
rack1
Rack ID 24_rack1 ['1861.421022']
rack1
Rack ID 24_rack1 ['1529.352055']
rack1
Rack ID 24_rack2 ['1940.722696']
rack2
Rack ID 24_rack2 ['1823.9197

['Aspirating 721.8710495 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 721.8710495 at <Deck><Slot A1><Container R_1><Well A1>',
 'Aspirating 721.8710495 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 721.8710495 at <Deck><Slot A1><Container R_1><Well A1>',
 'Aspirating 668.9573715 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 668.9573715 at <Deck><Slot A1><Container R_1><Well A2>',
 'Aspirating 668.9573715 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 668.9573715 at <Deck><Slot A1><Container R_1><Well A2>',
 'Aspirating 768.3775335 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 768.3775335 at <Deck><Slot A1><Container R_1><Well A3>',
 'Aspirating 768.3775335 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 768.3775335 at <Deck><Slot A1><Container R_1><Well A3>',
 'Aspirating 810.354098 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 810.354098 at <

In [2]:
'''OT1 -Single channel- Making up of main reagent stock solution (2 maximum) using a defined solvent and transfer onto 3mL fluidx vials'''

from opentrons import robot, containers, instruments
robot.head_speed(x=21000,  y=21000,  z=5000, a=700, b=700)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
stock_reagent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\stock_reagents.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\solvents.csv")

robot.reset()
"""Function that does two liquid handling transfers. First it will dilute a solid reagent in a big trough to the right concentration.
(Up to 2 reagents). Second, it will transfer the reagent from the big trough to the Fluix 24 vial rack, using the volume from the csv file
It requires 2 csv files. The first is the solvent csv, the second the custom made stock reagent csv."""
def stock_solution (amine, solvent):

    #Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough4row = containers.load("trough-12row", "C2")
    destination_stock = containers.load("FluidX_24_5ml", "A1", "stock")
    trash = containers.load("point", "C3")
    #Pipettes SetUp
    p1000 = instruments.Pipette(
        name= 'eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    id_header = "CPD ID"    
    solvent = "MeCN"
    stock_sol1 = "stock reagent 1"
    stock_sol2 = "stock reagent 2"
    location_header = "Location_trough"
    destination_location_header = "Location"
    volume_stock_header = "Volume to dispense (uL)"
    volume_per_vial = "Volume to dispense"
    
    for i, x in enumerate(solvent_df[id_header].tolist()):
        if x == solvent:
            solvent_location = solvent_df[location_header].tolist()[i]
        if x == stock_sol1:
            stock_sol1_loc = solvent_df[location_header].tolist()[i]
            stock_sol1_volume = solvent_df[volume_stock_header].tolist()[i]
        if x == stock_sol2:
            stock_sol2_loc = solvent_df[location_header].tolist()[i]
            stock_sol2_volume = solvent_df[volume_stock_header].tolist()[i]
    
    #Using the desired solvent, dilution of reagents 1 and/or 2 to the desired conc, in the big trough
    p1000.pick_up_tip()
    p1000.transfer([stock_sol1_volume], source_trough4row.wells(solvent_location), source_trough4row.wells(stock_sol1_loc).top(-5), new_tip = 'never')
    p1000.drop_tip()
    p1000.pick_up_tip()
    p1000.transfer([stock_sol2_volume], source_trough4row.wells(solvent_location), source_trough4row.wells(stock_sol2_loc).top(-5), new_tip = 'never')
    p1000.drop_tip()
    
    #robot.pause()
    
    #Reagents 1 and/or 2 are transfered to the 24 Fluidx vial rack.
    for i, x in enumerate(stock_reagent_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [stock_reagent_df[volume_per_vial].tolist()[i]]
        stock_id = stock_reagent_df[id_header].tolist()[i]
        if stock_id == stock_sol1:
            p1000.pick_up_tip()
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol1_loc), destination_stock.wells(destination_location).top(-5), new_tip = 'never')
            p1000.drop_tip()
        if stock_id == stock_sol2:
            p1000.pick_up_tip()
            #print ('incorrect')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol2_loc), destination_stock.wells(destination_location).top(-5), new_tip = 'never')   
            p1000.drop_tip()
    robot.home()

stock_solution (stock_reagent_df, solvent_df)

robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A8>',
 'Dispensing 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot 

In [64]:
from opentrons import robot, containers, instruments
robot.head_speed(x=16000, y=16000, z=3000, a=700, b=700)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
reaction_conditions_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\reaction_conditions.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\solvents.csv")

robot.reset()
# Deck setup
tiprack_300 = containers.load("tiprack-300ul", "D3")
source_trough12row = containers.load('trough-12row', "E2")
reaction_rack = containers.load("StarLab_96_tall", "D1")
trash = containers.load("point", "C3")

# Pipettes SetUp
p300_multi = instruments.Pipette(
    name='dlab_300multi',
    axis="a",
    trash_container=trash,
    tip_racks=[tiprack_300],
    max_volume=300,
    min_volume=30,
    channels=8,
)

cpd_id_heading = "CPD ID"
base = "DIPEA"
coupling = "T3P"
location_heading = "Location_trough"
reaction_heading = "reaction"
volume_coupling_heading = "Volume to dispense Coupling agent"
volume_base_heading = "Volume to dispense Base"
number_rows_heading = "Number rows"
for index, value in enumerate(solvent_df[cpd_id_heading].tolist()):
    if value == base:
        base_location = solvent_df[location_heading].tolist()[index]
    if value == coupling:
        coupling_location = solvent_df[location_heading].tolist()[index]        
for index, value in enumerate(reaction_conditions_df[reaction_heading].tolist()):
    if value == "Coupling_standard":
        base_volume = float(reaction_conditions_df[volume_base_heading].tolist()[index])
        coupling_volume = float(reaction_conditions_df[volume_coupling_heading].tolist()[index])
        number_rows = int(reaction_conditions_df[number_rows_heading].tolist()[index])
print(base_location, type(base_location), base_volume, type(base_volume))
p300_multi.distribute(base_volume, source_trough12row.wells(base_location),
                              [x.top() for x in reaction_rack.rows(0, to=number_rows)])
p300_multi.distribute(coupling_volume, source_trough12row.wells(coupling_location),
                              [x.top() for x in reaction_rack.rows(0, to=number_rows)])
robot.home()

robot.commands()
#DONE!!


A3 <class 'str'> 50.94703504 <class 'float'>


['Picking up tip from <WellSeries: <Well A1><Well B1><Well C1><Well D1><Well E1><Well F1><Well G1><Well H1>>',
 'Aspirating 284.7351752 at <Deck><Slot E2><Container trough-12row><Well A3>',
 'Dispensing 50.94703504 at <WellSeries: <Well A1><Well B1><Well C1><Well D1><Well E1><Well F1><Well G1><Well H1>>',
 'Dispensing 50.94703504 at <WellSeries: <Well A2><Well B2><Well C2><Well D2><Well E2><Well F2><Well G2><Well H2>>',
 'Dispensing 50.94703504 at <WellSeries: <Well A3><Well B3><Well C3><Well D3><Well E3><Well F3><Well G3><Well H3>>',
 'Dispensing 50.94703504 at <WellSeries: <Well A4><Well B4><Well C4><Well D4><Well E4><Well F4><Well G4><Well H4>>',
 'Dispensing 50.94703504 at <WellSeries: <Well A5><Well B5><Well C5><Well D5><Well E5><Well F5><Well G5><Well H5>>',
 'Blowing out at <Deck><Slot C3><Container point><Well A1>',
 'Aspirating 284.7351752 at <Deck><Slot E2><Container trough-12row><Well A3>',
 'Dispensing 50.94703504 at <WellSeries: <Well A6><Well B6><Well C6><Well D6><Well E6

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Dispensing 1000.0 at <Deck><Slot A1><Container stock><Well A1>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Dispensing 1000.0 at <Deck><Slot A1><Container stock><Well A1>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well B1>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Dispensing 1000.0 at <Deck><Slot A1><Container stock><Well A2>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Dispensing 1000.0 at <Deck><Slot A1><Container stock><Well A2>',
 'Drop_tip at <Deck><Slot C3><Container point><Well A1>',
 'Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well C1>',
 'Aspirating 1000.0 at <Deck><Slot C2><Container trough-12row><Well A2>',
 'Dispensing 10

In [ ]:
    robot.pause()
    for i, x in enumerate(amine_df[destination_location_header].tolist()):
        destination_location = x
        vol_to_dispense = [amine_df[volume_per_vial].tolist()[i]]
        amine_id = amine_df[code_header].tolist()[i]
        p1000.pick_up_tip()
        if amine_id == stock1:
            #print ('correct')
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol1_loc), destination_amine_stock.wells(destination_location).top(-5), new_tip = 'never')   
        if intermediate_id == stock2:
            #print ('incorrect')
            p1000.drop_tip()
            p1000.pick_up_tip()
            if vol_to_dispense != 0:
                p1000.transfer(vol_to_dispense, source_trough4row.wells(stock_sol2_loc), destination_amine_stock.wells(destination_location).top(-5), new_tip = 'never')   

        p1000.drop_tip()
        #else:
         #   print ("error")
          #  break

        #print(destination_location,vol_to_dispense,intermediate_id, stock1)
    robot.home()
stock_solution(amine_df, solvent_df)
#robot.commands()

In [37]:
from opentrons import robot, containers, instruments
robot.head_speed(x=18000,  y=18000,  z=5000, a=250, b=250)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
reaction_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\stock_reagents.csv")
#solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\solvents.csv")
robot.reset()

def intermediate_transfer (reaction, solvent):
    #Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough12row = containers.load("trough-12row", "C1")
    location_stock = containers.load("FluidX_24_5ml", "A1", "int")
    reaction_rack = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")
    #Pipettes SetUp
    p1000 = instruments.Pipette(
        name= 'eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    location_header = "Location"
    nb_reaction_header = "Number of reactions"
    volume_per_reaction_header = "Volume_stock per reaction"
    volume_max_header = "Volume max per vial"
    nb_reactions = int(reaction_df[nb_reaction_header].tolist()[1])
    volume_per_reaction = int(reaction_df[volume_per_reaction_header].tolist()[1])
    reaction = 0
    for index, value in enumerate(reaction_df[location_header].tolist()):   
        if reaction < nb_reactions:
            nb_reaction_per_vial = int(float(reaction_df[volume_max_header].tolist()[index]) // nb_reactions)
            source_location = value
            if nb_reactions - reaction < nb_reaction_per_vial:
                if nb_reactions - reaction == 1:
                    p1000.distribute(volume_per_reaction, location_stock.wells(source_location), reaction_rack.wells(reaction).top())
                else:
                    p1000.distribute(volume_per_reaction, location_stock.wells(source_location), [x.top() for x in reaction_rack.wells(reaction, to= nb_reactions)])
            else:
                p1000.distribute(volume_per_reaction, location_stock.wells(source_location), [x.top() for x in reaction_rack.wells(reaction, to= reaction + nb_reaction_per_vial-1)])
            reaction = reaction + nb_reaction_per_vial
    robot.home()
intermediate_transfer(reaction_df, solvent_df)
robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 990.0 at <Deck><Slot A1><Container int><Well A1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well B1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well D1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well E1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well F1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well G1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well H1>',
 'Blowing out at <Deck><Slot C3><Container point><Well A1>',
 'Aspirating 990.0 at <Deck><Slot A1><Container int><Well A1>',
 'Dispensing 120.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A2>',
 'Dispensing 120.0 at <Deck><Slot D1><Container St

In [46]:
    
    reaction_index = 0
    
    while (reaction_index < nb_reactions):
        for
        
        print (reaction_index)
        reaction_index += 1
    print(nb_reactions,volume_per_reaction, type(nb_reactions))

        
    
    #for index, value in enumerate(reaction_df[location_header].tolist()):
            
intermediate_transfer(reaction_df, solvent_df)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96 120 <class 'int'>


In [ ]:
for index, value in enumerate(reaction_df[location_header].tolist()):
        
        
        
    
    n=0
    for i, x in enumerate(reaction_df[location_header].tolist()):
        
        for 
        
        nb_reaction = int(reaction_df[nb_reaction_header].tolist()[i])
        volume_per_reaction = int(reaction_df[volume_per_reaction_header].tolist()[i])
        source_location = reaction_df[location_header].tolist()[i]
        volume_list = []
        destination_list = []
        #for i in range (n+0, n+nb_reaction):
            #destination_list.append(i)
            #volume_list.append(volume_per_reaction)
        p1000.distribute(volume_per_reaction, location_int.wells(source_location), [x.top() for x in reaction_rack.wells(n+0, to=n + nb_reaction-1)])
        n = nb_reaction + n

intermediate_transfer(reaction_df, solvent_df)
robot.commands()

In [42]:
class Reaction(object):
    def __init__(self, number_reaction, volume_per_reaction):
        self.number_reaction = number_reaction
        self.volume_per_reaction = volume_per_reaction
        self.locations = []
    def add_location(self, location):
        self.locations.append(location)
        

        
test = Reaction(96, 100)
test2 = Reaction(1313, 21435)

test.add_location("A1")
test2.add_location("A2")
#test2 = volume_per_reaction(100)
#print(Reaction.number_reaction)
print(Reaction)
data = test.locations

<class '__main__.Reaction'>


['A1']

In [ ]:
        

class Reactant(object):
    name = "main_sm"
    def __init__(self, volume_per_reaction, location)
    
number = num
    

In [23]:
class Car(object):
    wheels = 4
    def __init__(self, make):
        self.make = make

newCar = Car("Honda")
print ("My new car is a {}".format(newCar.make))
print ("My car, like all cars, has {} wheels".format(newCar.wheels))
print(newCar.wheels)
print(Car.wheels)

My new car is a Honda
My car, like all cars, has 4 wheels
4
4


In [32]:
class Dog:

    def __init__(self, name):
        self.name = name
        self.tricks = []    # creates a new empty list for each dog

    def add_trick(self, trick):
        self.tricks.append(trick)

d = Dog("Jose")
e = Dog("Rob")
e.add_trick("jump")
d.add_trick("bully")
d.add_trick("rob")
e.tricks

['jump']

In [ ]:
>>> d = Dog('Fido')
>>> e = Dog('Buddy')
>>> d.add_trick('roll over')
>>> e.add_trick('play dead')
>>> d.tricks                # unexpectedly shared by all dogs
['roll over', 'play dead']

In [7]:
class Dog:
    kind = 'canine'         # class variable shared by all instances
    def __init__(self, name):
        self.name = name    # instance variable unique to each instance
d = Dog("berti")
d.name
class Bike(object):
    #constructor
    def __init__(self, gear, speed, make):
        self.gear = gear
        self.speed = speed
        self.make = make

    def apply_brake(decrement):
        self.speed -= decrement

    def speed_up(increment):
        self.speed += increment
        
newbike = Bike("tesla")
print("Newbike is a {}".format(newbike.make))

TypeError: __init__() missing 2 required positional arguments: 'speed' and 'make'

In [13]:
class MyClass:
    """A simple example class"""
    i = 12345

    def f(self):
        return 'hello world'
    
print(MyClass.f)

<function MyClass.f at 0x000002A7CD8C5F28>


In [10]:
class Complex:
    def __init__(self, realpart, imagpart):
        self.r = realpart
        self.i = imagpart
x = Complex(3.0, -4.5)
x.r, x.i

(3.0, -4.5)

In [16]:
class Dog:
    kind = 'canine'         # class variable shared by all instances
    def __init__(self, name):
        self.name = name    # instance variable unique to each instance
d = Dog("berti")
d.name

'berti'

In [ ]:
>>> d = Dog('Fido')
>>> e = Dog('Buddy')
>>> d.kind                  # shared by all dogs
'canine'
>>> e.kind                  # shared by all dogs
'canine'
>>> d.name                  # unique to d
'Fido'
>>> e.name                  # unique to e
'Buddy'

In [3]:
class Car(object):
    wheels = 4

def __init__(self, make):
    self.make = make

newCar = Car("Honda")
print ("My new car is a {}".format(newCar.make))
print ("My car, like all cars, has {%d} wheels".format(Car.wheels))

TypeError: object() takes no parameters

In [4]:
reaction = 0
while (reaction < 96):
    print (reaction)
    reaction = reaction + 1
print("goodbye")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
goodbye


In [3]:
count = 0
while (count < 9):
   print ('The count is:', count)
   count = count + 1

print ("Good bye!")

The count is: 0
The count is: 1
The count is: 2
The count is: 3
The count is: 4
The count is: 5
The count is: 6
The count is: 7
The count is: 8
Good bye!


In [3]:
    for index, value in enumerate(reaction_df[location_header].tolist()):
        
        
        
    
    n=0
    for i, x in enumerate(reaction_df[location_header].tolist()):
        
        for 
        
        nb_reaction = int(reaction_df[nb_reaction_header].tolist()[i])
        volume_per_reaction = int(reaction_df[volume_per_reaction_header].tolist()[i])
        source_location = reaction_df[location_header].tolist()[i]
        volume_list = []
        destination_list = []
        #for i in range (n+0, n+nb_reaction):
            #destination_list.append(i)
            #volume_list.append(volume_per_reaction)
        p1000.distribute(volume_per_reaction, location_int.wells(source_location), [x.top() for x in reaction_rack.wells(n+0, to=n + nb_reaction-1)])
        n = nb_reaction + n

intermediate_transfer(reaction_df, solvent_df)
robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 990.0 at <Deck><Slot A1><Container int><Well C1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well B1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well D1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well E1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well F1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well G1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well H1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A2>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well B2>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C2>',
 'Dispensing 60.0 at <Deck><Slot D

In [26]:
#!/usr/bin/env python
# coding: utf-8

# In[22]:


from opentrons import robot, containers, instruments
robot.head_speed(x=18000,  y=18000,  z=5000, a=250, b=250)

class Vector(object):
    def tolist(self):
        return list(self.input_list)

    def astype(self, input_type):
        if input_type == int:
            return Vector([int(float(x)) for x in self.input_list])
        return Vector([input_type(x) for x in self.input_list])

    def __init__(self, input_list):
        self.input_list = input_list

class DataFrame(object):
    def __len__(self):
        return self.length

    def __getitem__(self, value):
        return Vector(self.dict_input[value])

    def __init__(self, dict_input, length):
        self.dict_input = dict_input
        self.length = length
#Function that reads a csv file correctly without having to import anything (issues with molport). Uses 2 classes, Vector and DataFrame
def read_csv(input_file):
    lines = open(input_file).readlines()
    header = lines[0].rstrip().split(",")
    out_d = {}
    for head in header:
        out_d[head] = []
    for line in lines[1:]:
        spl_line = line.rstrip().split(",")
        for i, head in enumerate(header):
            out_d[head].append(spl_line[i])
    df = DataFrame(out_d, len(lines[1:]))
    return df

#CSV file data
reaction_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\reaction_step1_intermediate2_MeI.csv")
solvent_df = read_csv(r"C:\Users\sdi35357\CODING\github_repo\OT1-coding\coupling_phip\csv\05-19_rd2\solvents.csv")
robot.reset()

def intermediate_transfer (reaction, solvent):
    #Deck setup
    tiprack_1000 = containers.load("tiprack-1000ul-H", "B3")
    source_trough12row = containers.load("trough-12row", "C1")
    location_int = containers.load("FluidX_24_5ml", "A1", "int")
    reaction_rack = containers.load("StarLab_96_tall", "D1")
    trash = containers.load("point", "C3")
    #Pipettes SetUp
    p1000 = instruments.Pipette(
        name= 'eppendorf1000',
        axis='b',
        trash_container=trash,
        tip_racks=[tiprack_1000],
        max_volume=1000,
        min_volume=30,
        channels=1,
    )
    
    location_header = "Location"
    #stock1 = "ImidInt-1"
    #stock2 = "ImidInt-2"
    #code_header = "Code"
    nb_reaction_header = "number of reaction"
    volume_per_reaction_header = "Volume_stock per reaction"
    
    n=0
    for i, x in enumerate(reaction_df[location_header].tolist()):
        
        nb_reaction = int(reaction_df[nb_reaction_header].tolist()[i])
        volume_per_reaction = int(reaction_df[volume_per_reaction_header].tolist()[i])
        source_location = reaction_df[location_header].tolist()[i]
        volume_list = []
        destination_list = []
        #for i in range (n+0, n+nb_reaction):
            #destination_list.append(i)
            #volume_list.append(volume_per_reaction)
        p1000.distribute(volume_per_reaction, location_int.wells(source_location), [x.top() for x in reaction_rack.wells(n+0, to=n + nb_reaction-1)])
        n = nb_reaction + n

intermediate_transfer(reaction_df, solvent_df)
robot.commands()

['Picking up tip from <Deck><Slot B3><Container tiprack-1000ul-H><Well A1>',
 'Aspirating 990.0 at <Deck><Slot A1><Container int><Well C1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well B1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well D1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well E1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well F1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well G1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well H1>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well A2>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well B2>',
 'Dispensing 60.0 at <Deck><Slot D1><Container StarLab_96_tall><Well C2>',
 'Dispensing 60.0 at <Deck><Slot D